In [2]:
import torch
from IPython.display import Image  
import os 
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from tqdm import tqdm
from xml.dom import minidom
import numpy as np
import matplotlib.pyplot as plt
import random
from PIL import Image, ImageDraw
import shutil

In [3]:
def read_anno(xml_file):
    root = ET.parse(xml_file).getroot()
    
    info_dict = {}
    info_dict['bboxes'] = []

    for elem in root:
        if elem.tag == "filename":
            info_dict['filename'] = elem.text
            
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))
            
            info_dict['image_size'] = tuple(image_size)
       
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text
                    
                elif subelem.tag == "bndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(subsubelem.text)            
            info_dict['bboxes'].append(bbox)
    
    return info_dict

In [18]:
print(read_anno('D:/Research Paper/Teeth Paper/Teeth-Extraction-master/annotations/24.xml'))

{'bboxes': [{'class': 'normal', 'xmin': 2160, 'ymin': 592, 'xmax': 2333, 'ymax': 821}, {'class': 'normal', 'xmin': 1820, 'ymin': 681, 'xmax': 1880, 'ymax': 881}, {'class': 'normal', 'xmin': 1738, 'ymin': 654, 'xmax': 1831, 'ymax': 894}, {'class': 'normal', 'xmin': 1640, 'ymin': 654, 'xmax': 1756, 'ymax': 896}, {'class': 'normal', 'xmin': 1536, 'ymin': 670, 'xmax': 1638, 'ymax': 894}, {'class': 'normal', 'xmin': 1462, 'ymin': 674, 'xmax': 1536, 'ymax': 876}, {'class': 'normal', 'xmin': 953, 'ymin': 545, 'xmax': 1098, 'ymax': 792}, {'class': 'normal', 'xmin': 1098, 'ymin': 583, 'xmax': 1236, 'ymax': 841}, {'class': 'normal', 'xmin': 1236, 'ymin': 596, 'xmax': 1302, 'ymax': 870}, {'class': 'normal', 'xmin': 1391, 'ymin': 654, 'xmax': 1462, 'ymax': 865}, {'class': 'normal', 'xmin': 1976, 'ymin': 625, 'xmax': 2062, 'ymax': 852}, {'class': 'normal', 'xmin': 1898, 'ymin': 907, 'xmax': 2016, 'ymax': 1103}, {'class': 'normal', 'xmin': 1827, 'ymin': 912, 'xmax': 1907, 'ymax': 1134}, {'class': 'n

In [7]:
class_name_to_id_mapping = {"normal": 0, "class 1": 1, "class 2": 2, "class 3": 3, "class 4": 4, "class 5": 5, "class 6": 6}

In [9]:
info_dict = read_anno('D:/Research Paper/Teeth Paper/Teeth-Extraction-master/annotations/22.xml')

In [10]:
info_dict

{'bboxes': [{'class': 'normal',
   'xmin': 2236,
   'ymin': 761,
   'xmax': 2505,
   'ymax': 1038},
  {'class': 'normal', 'xmin': 2051, 'ymin': 796, 'xmax': 2240, 'ymax': 1078},
  {'class': 'normal', 'xmin': 1978, 'ymin': 823, 'xmax': 2089, 'ymax': 1072},
  {'class': 'normal', 'xmin': 1882, 'ymin': 827, 'xmax': 1985, 'ymax': 1085},
  {'class': 'normal', 'xmin': 1799, 'ymin': 842, 'xmax': 1888, 'ymax': 1100},
  {'class': 'normal', 'xmin': 1722, 'ymin': 841, 'xmax': 1796, 'ymax': 1116},
  {'class': 'normal', 'xmin': 1651, 'ymin': 841, 'xmax': 1727, 'ymax': 1132},
  {'class': 'normal', 'xmin': 1582, 'ymin': 832, 'xmax': 1662, 'ymax': 1138},
  {'class': 'normal', 'xmin': 1500, 'ymin': 836, 'xmax': 1576, 'ymax': 1107},
  {'class': 'normal', 'xmin': 1418, 'ymin': 845, 'xmax': 1500, 'ymax': 1110},
  {'class': 'normal', 'xmin': 1313, 'ymin': 843, 'xmax': 1405, 'ymax': 1092},
  {'class': 'normal', 'xmin': 1227, 'ymin': 832, 'xmax': 1331, 'ymax': 1105},
  {'class': 'normal', 'xmin': 1075, 'ymin'

In [12]:
def convert_to_yolov3(info_dict):
    print_buffer = []
    
    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())
        
        # Transform the bbox co-ordinates as per the format required by YOLO v3
        b_center_x = (b["xmin"] + b["xmax"]) / 2 
        b_center_y = (b["ymin"] + b["ymax"]) / 2
        b_width    = (b["xmax"] - b["xmin"])
        b_height   = (b["ymax"] - b["ymin"])
        
        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]  
        b_center_x /= image_w 
        b_center_y /= image_h 
        b_width    /= image_w 
        b_height   /= image_h 
        
        #Write the bbox details to the file 
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))
        
    # Name of the file which we have to save 
    save_file_name = os.path.join("annotations", info_dict["filename"].replace("bmp", "txt"))
    
    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [14]:
info_dict = read_anno('D:/Research Paper/Teeth Paper/Teeth-Extraction-master/annotations/22.xml')
convert_to_yolov5(info_dict)

FileNotFoundError: [Errno 2] No such file or directory: 'annotations\\22.txt'

In [13]:
for i in tqdm(range(int(9e6))):
    pass

100%|██████████| 9000000/9000000 [00:04<00:00, 1897816.15it/s]


In [17]:
os.path.join("annotations", info_dict["filename"].replace("bmp", "txt"))

'annotations\\22.txt'